In [ ]:
import os
import openai

client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [ ]:
response = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    # Write your prompt
    prompt="In two sentences, how can the OpenAI API be used to upskill myself?",
    max_tokens=200
)

print(response.choices[0].text)

In [ ]:
# Converts the response into a dictionary
response.model_dump()

In [ ]:
# Using an organization can help with limits, billing, etc. 

from openai import OpenAI

client = OpenAI(
  organization='org-4otiQhcm1LBuWdDYRBImSUlI',
  project='proj_bAW2JMzoEkGe09EcZZrTQ5oi',
)

In [ ]:
stream = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "Say this is a test"}],
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

# OpenAI Models

## Completions 

- Receive continuation of a prompt
- Single Turn Tasks
    - Answer quesions
    - Classification
    - Sentiment Analysis
    - Explain Complex Topics

When we send a text to a text completion endpoint, the api responds with the text that is *most likely* to complete the prompt. Responses are non-deterministic, though.

Sometimes this randomness is not desirable, like in a customer service chatbot. **Temperature** is a parameter ranging from 0 (highly deterministic) to 2 (very random). 

On top of answering questions, other kinds of tasks where the completions model can help are summarization, text content generation and transformation (find and replace)...

The **max_tokens** param helps shape the size of the response and control the price.

Classification of sentences or tokens is another capability of the completions model.

Depending on the amount of examples provided, our requests can be categorized as:
- **Zero shot**: no example

These two are colled in-context learning:
- **One shot**: one example
- **Few shot**: several examples

In [ ]:
for temperature in [0,1]:
    response = client.completions.create(
        model="gpt-3.5-turbo-instruct",
        # Write your prompt
        prompt="In two sentences, how can the OpenAI API be used to upskill myself?",
        max_tokens=100, 
        temperature=temperature
    )
    
    print(f"Temperature {temperature}: {response.choices[0].text}")

## Chat 

- Multi-turn conversations
  - Ideation
  - Customer Support Assistant
  - Personal Tutor
  - Translations
  - Write code
- Also performs well single turn tasks

One benefit of the chat completions is that they offer better customization through the use of *roles*

Another benefit is the instruct models are more expensive than the chat ones.

### Roles 

- **System**: controls assistant's behavior
- **User**: instructs the assistant
- **Assistant**: response to the user instruction.
  - Can also be written by the user to provide examples



In [ ]:
stream = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "system", "content": "you are a data science tutor who speaks concisely"}, 
              {"role": "user", "content": "what is the difference between lasso and ridge"}, 
             ],
    max_tokens=200,
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

We can provide example messages as a conversation to the system so it can learn how do we want it to interact with us.

Storing responses allow us to create back and forth conversations. Thats how chatgpt works.

In [ ]:
response = client.chat.completions.create(
   model="gpt-4o-mini",
   # Add a user and assistant message for in-context learning
   messages=[
     {"role": "system", "content": "You are a helpful Python programming tutor."},
     {"role": "user", "content": "Explain python lists"},
     {"role": "assistant", "content": "Python list is a type of python ordered structure that can contain other objects."},
     {"role": "user", "content": "Explain what the type() function does."}
   ]
)

print(response.choices[0].message.content)

In [ ]:
messages = [{"role": "system", "content": "You are a helpful math tutor."}]
user_msgs = ["Explain what pi is.", "Summarize this in two bullet points."]

for q in user_msgs:
    print("User: ", q)
    
    # Create a dictionary for the user message from q and append to messages
    user_dict = {"role": "user", "content": q}
    messages.append(user_dict)
    
    # Create the API request
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        max_tokens=100
    )
    
    # Convert the assistant's message to a dict and append to messages
    assistant_dict = {"role": "assistant", "content": response.choices[0].message.content}
    messages.append(assistant_dict)
    print("Assistant: ", response.choices[0].message.content, "\n")

In [ ]:
# Initial system message to set the assistant's behavior
conversation = [
    {"role": "system", "content": "you are an experienced tourist guide with deep knowledge about Paris"}
]

# Function to add new messages to the conversation and get a response
def send_message(conversation, user_input):
    # Add user's message to the conversation
    conversation.append({"role": "user", "content": user_input})

    # Call the GPT-3.5 Turbo model
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",  # Use the GPT-3.5 Turbo model
        messages=conversation,  # Pass the conversation history
        temperature=0,          # Set temperature for deterministic output
        max_tokens=100          # Limit the response to 100 tokens
    )

    # Get assistant's response
    assistant_reply = response.choices[0].message.content
    
    # Add assistant's response to the conversation
    conversation.append({"role": "assistant", "content": assistant_reply})

    # Print the assistant's reply
    print(assistant_reply)

# List of user inputs
user_inputs = [
    "How far away is the Louvre from the Eiffel Tower (in miles) if you are driving?",
    "Where is the Arc de Triomphe?",
    "What are the must-see artworks at the Louvre Museum?"
]

# Send each message one by one
for user_input in user_inputs:
    send_message(conversation, user_input)

## Moderation 

- Checks content for violations of OpenAIs usage policies, including inciting violence or hate speech.
- The sensitivity can be customized

The categoryScores object of the response of the moderation model gives us fine details about different aspects of the moderation.

In [ ]:
import json 

# Create a request to the Moderation endpoint
response = client.moderations.create(
    model='text-moderation-latest',
    input='My favorite book is To Kill a Mockingbird.'
)

# Print the category scores
print(response.results[0].category_scores)

In [ ]:
print(json.dumps(response.results[0].category_scores, indent=4))

# Speech to Text Transcription with Whisper 

Speech to text capabilities: 
- Transcribe audio
- Translate and transcribe audio into English

The quality of the translation depends on the audio quality, the audio language and model's knowledge of the subject matter.

In [ ]:
audio = open('11 oct 9.53_.mp3', 'rb')

# Create a request to the Moderation endpoint
response = client.audio.transcriptions.create(
    model='whisper-1',
    file=audio
)

print(response)

In [ ]:
audio = open('flas-christmas-sp.mp3', 'rb')

# Create a request to the Moderation endpoint
response = client.audio.translations.create(
    model='whisper-1',
    file=audio
)

print(response)

In order to help the quality of the transcription we can provide an optional prompt indicating the style of the output we expect and context for the transcript.

In [ ]:
audio = open('flas-christmas-sp.mp3', 'rb')

# Create a request to the Moderation endpoint
response = client.audio.translations.create(
    model='whisper-1',
    file=audio,
    prompt="[Music] In Spain, the whole family gathers on Dec. 24th to celebrate Christmas Eve."
)

print(response)

# Combining Models

Chaining models is feeding the output from one model into another 
Can use the same model multiple times
